### CUT AND SAVE IMAGES

In [1]:
import os
import torch
import torch.nn as nn
import random
import math
import numpy as np
import torchvision 
from torchvision import transforms, datasets
import torchvision.datasets as dataset
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pandas as pd
import time
import glob
import torch.optim as optim
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
print(torch.__version__)

1.8.0


In [52]:
import torch
t = torch.tensor(np.random.random_sample((3,200,100)))
# functional.crop(t,100,200,200,200)
based_t = torch.zeros((3,500,300))
based_t[:,:200,:100] = t
print(based_t.size(1))
print(t.size(1))
print(based_t[0])

500
200
tensor([[0.9430, 0.6997, 0.5442,  ..., 0.0000, 0.0000, 0.0000],
        [0.9135, 0.8915, 0.7127,  ..., 0.0000, 0.0000, 0.0000],
        [0.9264, 0.9913, 0.9893,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


In [2]:
import torchvision.transforms
from torchvision.transforms import functional
from tqdm import tqdm
from PIL import Image
func_crop = functional.crop

def cut_img_png(img_path_list, save_path, stride=128):
    img_size = 256
    toimage = transforms.ToPILImage()
    totensor = transforms.ToTensor()
    os.makedirs(f'{save_path}{img_size}', exist_ok=True)
    num = 0
    for path in tqdm(img_path_list):
        img = Image.open(path)
        img = totensor(img)
        for top in range(0, img.size()[2], stride):
            for left in range(0, img.size()[1], stride):
                based_tensor = torch.zeros((3,img_size,img_size))
                arr_cropped = func_crop(img,top,left,img_size,img_size)
                w, h = arr_cropped.size(1), arr_cropped.size(2)
                based_tensor[:, :w, :h] = arr_cropped
                to_pil = toimage(based_tensor)
                to_pil.save(f'{save_path}{img_size}/{num}.png')
                num+=1
                
def cut_img(img_path_list, save_path, stride):
    img_size = 256
    os.makedirs(f'{save_path}{img_size}', exist_ok=True)
    num = 0
    for path in tqdm(img_path_list):
        img = cv2.imread(path)
        for top in range(0, img.shape[0], stride):
            for left in range(0, img.shape[1], stride):
                piece = np.zeros([img_size, img_size, 3], np.uint8)
                temp = img[top:top+img_size, left:left+img_size, :]
                piece[:temp.shape[0], :temp.shape[1], :] = temp
                np.save(f'{save_path}{img_size}/{num}.npy', piece)
                num+=1                

def cut_img_half(img_path_list, save_path):
    os.makedirs(f'{save_path}half', exist_ok=True)
    num = 0
    for path in tqdm(img_path_list):
        img = cv2.imread(path)
        len_half_x = img.shape[1]//2
        ratio_len = img.shape[0]/img.shape[1]
        len_half_y = img.shape[0]//2
        for top in range(0, img.shape[0], len_half_y//2):
            for left in range(0, img.shape[1], len_half_x//2):
                piece = np.zeros([len_half_y, len_half_x, 3], np.uint8)
                temp = img[top:top+len_half_y, left:left+len_half_x, :]
                piece[:temp.shape[0], :temp.shape[1], :] = temp
                np.save(f'{save_path}half/{num}.npy', piece)
                num+=1

def cut_img_6devide(img_path_list, save_path):
    from tqdm import tqdm
    os.makedirs(f'{save_path}6devide', exist_ok=True)
    num = 0
    for path in tqdm(img_path_list):
        img = cv2.imread(path)
        len_x = img.shape[1]//6
        len_y = img.shape[0]//6
        for top in range(0, img.shape[0], len_y//2):
            for left in range(0, img.shape[1], len_x//2):
                piece = np.zeros([len_y, len_x, 3], np.uint8)
                temp = img[top:top+len_y, left:left+len_x, :]
                piece[:temp.shape[0], :temp.shape[1], :] = temp
                np.save(f'{save_path}6devide/{num}.npy', piece)
                num+=1                
                

def cut_img_fullscreen(img_path_list, save_path):
    from tqdm import tqdm
    os.makedirs(f'{save_path}full', exist_ok=True)
    num = 0
    for path in tqdm(img_path_list):
        img = cv2.imread(path)
        len_x = img.shape[1]
        len_y = img.shape[0]
        np.save(f'{save_path}full/{num}_full.npy', img)
        num+=1                

In [5]:
# cut_img_png(train_input_files, './camera_dataset/train_input_img_png_',128)
# cut_img_png(train_label_files, './camera_dataset/train_label_img_png_')
cut_img_png(val_input_files, './camera_dataset/val_input_img_png_')
cut_img_png(val_label_files, './camera_dataset/val_label_img_png_')


100%|██████████████████████████████| 120/120 [24:05<00:00, 12.04s/it]


In [4]:
import pandas as pd
train_csv = pd.read_csv('./camera_dataset/train.csv')
test_csv = pd.read_csv('./camera_dataset/test.csv')
train_all_input_files = './camera_dataset/train_input_img/'+train_csv['input_img']
train_all_label_files = './camera_dataset/train_label_img/'+train_csv['label_img']
train_input_files = train_all_input_files[120:].to_numpy()
train_label_files = train_all_label_files[120:].to_numpy()

val_input_files = train_all_input_files[:120].to_numpy()
val_label_files = train_all_label_files[:120].to_numpy()

In [12]:
import numpy as np
print(train_input_files[0])

np.load('./camera_dataset/train_input_img_256/0.npy')#,allow_pickle = True)

./camera_dataset/train_input_img/train_input_10120.png


array([[[19, 21, 37],
        [19, 21, 37],
        [19, 18, 33],
        ...,
        [ 4, 13,  8],
        [ 6, 16, 10],
        [ 6, 15, 13]],

       [[19, 21, 37],
        [19, 21, 37],
        [19, 18, 33],
        ...,
        [ 4, 13,  8],
        [ 6, 16, 10],
        [ 6, 15, 13]],

       [[19, 18, 53],
        [19, 18, 53],
        [15, 19, 48],
        ...,
        [ 4, 12, 11],
        [ 8, 13, 13],
        [ 6, 15, 13]],

       ...,

       [[ 6,  7, 14],
        [ 6,  7, 14],
        [ 8,  6, 14],
        ...,
        [ 4, 14, 14],
        [ 2, 13, 17],
        [ 4, 13, 18]],

       [[ 6,  7, 11],
        [ 6,  7, 11],
        [ 6,  8, 11],
        ...,
        [ 6, 13, 18],
        [ 6, 13, 18],
        [ 2, 13, 18]],

       [[ 4,  7,  8],
        [ 4,  7,  8],
        [ 2,  9, 10],
        ...,
        [ 6, 13, 21],
        [ 4, 16, 20],
        [ 4, 16, 18]]], dtype=uint8)

In [ ]:
cut_img(train_input_files, './camera_dataset/train_input_img_')
cut_img(train_label_files, './camera_dataset/train_label_img_')
cut_img(val_input_files, './camera_dataset/val_input_img_')
cut_img(val_label_files, './camera_dataset/val_label_img_')
